In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [13]:
import matplotlib.pyplot as plt

**Importing train data**

In [14]:
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

In [15]:
train.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float64(135), int64(3)
memory usage: 2.5 GB


**Checking nulls in train data**

In [17]:
nulls = train.columns[train.isna().any()].tolist()
train_null = train[nulls]
nulls, len(nulls)

(['feature_3',
  'feature_4',
  'feature_7',
  'feature_8',
  'feature_9',
  'feature_10',
  'feature_11',
  'feature_12',
  'feature_13',
  'feature_14',
  'feature_15',
  'feature_16',
  'feature_17',
  'feature_18',
  'feature_19',
  'feature_20',
  'feature_21',
  'feature_22',
  'feature_23',
  'feature_24',
  'feature_25',
  'feature_26',
  'feature_27',
  'feature_28',
  'feature_29',
  'feature_30',
  'feature_31',
  'feature_32',
  'feature_33',
  'feature_34',
  'feature_35',
  'feature_36',
  'feature_44',
  'feature_45',
  'feature_55',
  'feature_56',
  'feature_58',
  'feature_59',
  'feature_72',
  'feature_73',
  'feature_74',
  'feature_75',
  'feature_76',
  'feature_78',
  'feature_79',
  'feature_80',
  'feature_81',
  'feature_82',
  'feature_84',
  'feature_85',
  'feature_86',
  'feature_87',
  'feature_88',
  'feature_90',
  'feature_91',
  'feature_92',
  'feature_93',
  'feature_94',
  'feature_96',
  'feature_97',
  'feature_98',
  'feature_99',
  'feature_10

In [18]:
#fig = plt.figure(figsize = (30,50))
#ax = fig.gca()
#train_null.hist(ax = ax, bins=30)

**Filling the nulls with mode**

In [19]:
fill_mode = lambda col: col.fillna(col.mode()[0])

In [20]:
train = train.apply(fill_mode, axis=0)
train.isna().sum()

date           0
weight         0
resp_1         0
resp_2         0
resp_3         0
              ..
feature_126    0
feature_127    0
feature_128    0
feature_129    0
ts_id          0
Length: 138, dtype: int64

**Checking cumulative sum of different resp over time**

In [21]:
#fig = plt.figure(figsize=(16,6))
#ax = plt.subplot(1,1,1)
#train.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']].sum().cumsum().plot(ax=ax)
#plt.title('Cumulative Sum of Different RESP\'s',fontsize=18)
#plt.xlabel('Date',fontsize=14)
#plt.axvspan(0,92,linestyle=':',linewidth=2,label='first 92 days',color='darkorange',alpha=.2)
#plt.legend(fontsize=12,ncol=3,loc=2);

**Since we have an high increase in the first 92 days, it could be a period of assestment of the model so we can drop them**<br>
**We also drop the rows that have 'wheight' = 0, since they will not be used for evaluation**

In [22]:
train = train.query('date > 92')
train = train[train['weight'] > 0]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1549854 entries, 553726 to 2390489
Columns: 138 entries, date to ts_id
dtypes: float64(135), int64(3)
memory usage: 1.6 GB


**We define a target column ('Action') multypling the weight by the average of all the resp**

In [23]:
train['action'] = ((train['weight'].values * \
                         (train['resp_1'] + train['resp_2'] + train['resp_3'] + train['resp_4'] + train['resp']).values)/5 > 0).astype('int')
train['action']

553726     1
553728     1
553729     0
553730     1
553732     1
          ..
2390444    0
2390446    0
2390478    0
2390481    0
2390489    0
Name: action, Length: 1549854, dtype: int64

**Since we summarize all the resp in the action column, we can drop them**

In [24]:
train = train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4'], axis=1)

**Defining X and Y**<br>
We keep date below 450 to train, and above to validate

In [25]:
#X = train.drop(['date', 'ts_id', 'action'], axis=1)
X_train = train.loc[train.date < 425].drop(['date', 'ts_id', 'action'], axis=1)
X_val = train.loc[train.date >= 425].drop(['date', 'ts_id', 'action'], axis=1) 
#y = train['action']
y_train = train.loc[train.date < 425]['action']
y_val = train.loc[train.date >= 425]['action']


**scale with quantile transformer** n_quantiles = 100, output_distribution = normal

In [26]:
from sklearn.preprocessing import QuantileTransformer

quant_transf = QuantileTransformer(n_quantiles=100, output_distribution='normal')
X_train = quant_transf.fit_transform(X_train)
X_val = quant_transf.transform(X_val)

**Getting prediction with a base model**

In [28]:
from sklearn.svm import LinearSVC

svc = LinearSVC()

svc.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [31]:
y_pred = svc.predict(X_val)
svc.score(X_val, y_val)

ValueError: Found input variables with inconsistent numbers of samples: [339964, 1209890]